In [ ]:
from recognition import solver
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# global variables
BATCH_SIZE = 128 # batch size
IMAGE_SIZE = 128 # image size
COLOR_MODE = 'rgb' # color mode 

train_generator = solver.my_generator('dataset/train', 
                                       rescale=1./255,
                                       # horizontal_flip=True, 
                                       zoom_range=0.2,
                                       shuffle=True,
                                       batch_size=BATCH_SIZE, 
                                       color_mode=COLOR_MODE, 
                                       target_size=(IMAGE_SIZE, IMAGE_SIZE))

valid_generator = solver.my_generator('dataset/validation', 
                                       rescale=1./255,
                                       shuffle=True,
                                       batch_size=BATCH_SIZE, 
                                       color_mode=COLOR_MODE, 
                                       target_size=(IMAGE_SIZE, IMAGE_SIZE))

print('Writing the class_indices into csv file.')
solver.write_into_csv(train_generator.class_indices, csv_path='class_indices.csv')

# VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Activation, Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, PReLU
from keras.models import Model
from keras.regularizers import l2


def build_model_based_on_vgg16(out_dims, base_weights=None, 
                               input_shape=(224, 224, 3), weights=None):
    """
    Inputs: 
      - out_dims:
      - base_weights: The weights of the vgg16 model
      - input_shape:
      - weights: The weights of the model which based on vgg16
      
    Return:
       - model: 
    """
    vgg16_base_model = VGG16(weights=base_weights, include_top=False, 
                             input_shape=input_shape)
    x = vgg16_base_model.output
    x = Flatten()(x)

    x = Dense(2048)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')
    x = Dropout(0.6)(x)
    
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')
    x = Dropout(0.6)(x)
    
    x = Dense(out_dims, kernel_regularizer=l2(0.01))(x)
    x = Activation('softmax')(x)
    model = Model(inputs=vgg16_base_model.input, outputs=x)
    
    if weights:
        model.load_weights(weights)

    return model

solver.clear_session()
channels = solver.get_channels(COLOR_MODE)
model = build_model_based_on_vgg16(100, base_weights='imagenet', 
                                   input_shape=(IMAGE_SIZE, IMAGE_SIZE, channels), 
                                   weights=None)
for index, layer in enumerate(model.layers):
    print(index, layer.name)
    
print('\n The architecture of the model:')
model.summary()

# Freeze some layers

In [ ]:
model = solver.freeze_layers(model, index=10)
solver.count_parameters(model) 

# Train model

In [ ]:
LEARNING_RATE = 0.01 # learning rate
DECAY = 1e-3 # decay 
EPOCHS = 50 # epochs

model_path = 'models/VGG16/model.h5'
history = solver.train_model(model, train_generator, 
                             valid_generator, 
                             model_path=model_path, 
                             batch_size=BATCH_SIZE, 
                             learning_rate = LEARNING_RATE, 
                             decay=DECAY, epochs=EPOCHS)

# Draw history curve

In [ ]:
solver.draw_plot(history)

# Evaluate the performance of our model

In [ ]:
generator = solver.my_generator('dataset/validation', 
                                rescale=1./255, 
                                shuffle=False,
                                batch_size=BATCH_SIZE, 
                                target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                color_mode=COLOR_MODE)

prediction = solver.get_prediction(model, generator)

for k in [1, 5]:
    correct_number, total_number = solver.evaluate_topk_accuracy(generator, 
                                                                 prediction=prediction, 
                                                                 k=k)

# Only use this model to predict

In [ ]:
class_indices = solver.get_class_indices('class_indices.csv')
print(class_indices)

In [ ]:
csv_data = solver.get_csv_format_data(generator, class_indices, 
                                      topk_indices=topk_indices)
solver.write_into_csv(csv_data, 'results.csv')